#### Importing all the packages

In [1]:
import os 
import sys

sys.path.insert(0, 'icenumerics/')

import icenumerics as ice
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

ureg = ice.ureg
idx = pd.IndexSlice

import warnings
warnings.filterwarnings("error")


%reload_ext autoreload
%autoreload 2

import magcolloids as mgc

mpl.rc('text', usetex=False)
mpl.rcParams['figure.dpi'] = 100

mpl.rcParams['font.family'] = 'serif'

from multiprocessing import Pool
import tqdm.auto as tqdm

#### Importing data

In [2]:
# < This function define the time/frame where the loop start and end >

def time(n):
    
    B_max = 133
    B_min = 105
    slope = 0.069
    tim = []
    t = [round(B_max/slope)]
    
    for i in range(1,n):
        t.append(round(((B_max-B_min)/slope+t[i-1])))
    for i in range(len(t)):
        tim.append(t[i]*100)
    return tim

In [3]:
# < This function reorder the ids in order to have the particle and the corresponding trap with the same id value >

def reordering_id(df):
    df = df.reset_index()
    df["id"] = df["id"]-min(df["id"])
    df = df.set_index(['frame', 'id'])
    del df['type']
    return df

In [4]:
# < This function filter the df values that match with the time/frame defined on function time >

def filter_RPM_data(df,t):
    df_def = []
        
    for i in range(len(t)):
        if i % 2 == 0: # if i is even 
            
            df_def.append(df.loc[[t[i]]]) # To save df at higher magnetic field value mT
        else:
            pass
#            df_def.append(df.loc[[t[i]]])  # To save df at lower magnetic field value mT

    df_f = pd.concat(df_def, axis =0)
    return df_f

In [5]:
# < Clean df >

def del_col(df):
    del df["mux"]
    del df["muy"]
    del df["muz"]

    del df["tmux"]
    del df["tmuy"]
    del df["tmuz"]
    return df

In [6]:
def analysis(exp_entry):
    
    k = exp_entry[1].exp
    
    # < Load .lammpstrj data > 

    load_obj = mgc.trj_lazyread(r"/home/carolina/lammps_script-7/BidisperseColloidalIce_RPM_rate0,05_l22_exp"+str(k)+".lammpstrj", output = ["x","y","z","mux","muy","muz"])
    
    
    bnd = load_obj.get_bounds()
    trj = load_obj.read_trj()
    
    # < Separate the high susceptibility particles and low susceptibility particles >

    x = trj.loc[[883000]]
    y = x.loc[(x['muz'] > 0.0003), 'type'].unique()
    z = x.loc[((x['muz'] > 0.00005) & (x['muz'] < 0.0003)), 'type'].unique()
    
    part1 = trj.loc[trj['type'] == y[0]]
    part2 = trj.loc[trj['type'] == z[0]]
    
    # < Separate between long and short traps >

    d = x.loc[(abs(x['mux']) > 10), 'type'].unique()
    e = x.loc[(abs(x['muy']) > 10), 'type'].unique()
    
    trap_long = trj.loc[trj['type'] == e[0]]
    trap_short = trj.loc[trj['type'] == d[0]]
    
    t = time(18)
    
    t_short = reordering_id(trap_short)
    t_long = reordering_id(trap_long)
    p1 = reordering_id(part1)
    p2 = reordering_id(part2)
    
    t_short = filter_RPM_data(t_short,t)
    t_long = filter_RPM_data(t_long,t)
    p1 = filter_RPM_data(p1,t)
    p2 = filter_RPM_data(p2,t)
    
    t_s = t_short.rename(columns={"x": "tx", "y": "ty", "z": "tz", "mux": "tmux", "muy": "tmuy", "muz" : "tmuz"})
    t_l = t_long.rename(columns={"x": "tx", "y": "ty", "z": "tz", "mux": "tmux", "muy": "tmuy", "muz" : "tmuz"})
    
    a  = pd.concat((p1,t_s), axis = 1)
    b  = pd.concat((p2,t_l), axis = 1)
    
    a = del_col(a)
    b = del_col(b)
    
    # < Define the positive spin if points towards x/y positives >
    
    a['resX'] = a["x"]-a["tx"]
    b['resY'] = b["y"]-b["ty"]
    
    a["spin"] = np.NaN
    b["spin"] = np.NaN
    a.loc[a.resX > 0, "spin"] = 1
    a.loc[a.resX < 0, "spin"] = -1
    b.loc[b.resY > 0, "spin"] = 1
    b.loc[b.resY < 0, "spin"] = -1
    
    a = a.drop(columns=["x", "tx", "y", "ty", "z", "tz", "resX"])
    b = b.drop(columns=["x", "tx", "y", "ty", "z", "tz", "resY"])
    
    
    # < Save data >
    
    df_short = filter_RPM_data(a,t)
    df_long = filter_RPM_data(b,t)
    
    save_dir = 'Data_RPM_rate0,05'
    
    df_short.to_csv(os.path.join(save_dir,'Spins_short_133mT_EXP'+str(k)), sep = '\t')
    df_long.to_csv(os.path.join(save_dir,'Spins_long_133mT_EXP'+str(k)), sep = '\t')
    

In [7]:
exp =  np.arange(25,50)
experiments = pd.DataFrame({"exp":exp.flatten()})

In [ ]:
if __name__ ==  '__main__': 
    num_processors = 10
    p=Pool(processes = num_processors)

        
    list(tqdm.tqdm(p.imap(analysis, experiments.iterrows()), total=len(experiments)))